# Import required libraries

In [116]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
from sklearn.feature_selection import SelectKBest, f_classif
import warnings
warnings.filterwarnings('ignore')

# Set up visualization style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# 1. Data Loading and Initial Exploration

In [117]:
# Load the dataset
df = pd.read_csv('heart.csv')

# Display basic information about the dataset
print("Dataset Shape:", df.shape)
print("\nFirst 5 rows:")
display(df.head())

print("\nDataset Info:")
display(df.info())

print("\nBasic Statistics:")
display(df.describe().round(2))

print("\nMissing Values:")
display(df.isnull().sum())
print(f"Total missing values: {df.isnull().sum().sum()}")
print(f"Total duplicates: {df.duplicated().sum().sum()}")

print("\nColumn Names:")
print(df.columns.tolist())


Dataset Shape: (1025, 14)

First 5 rows:


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0



Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1025 entries, 0 to 1024
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1025 non-null   int64  
 1   sex       1025 non-null   int64  
 2   cp        1025 non-null   int64  
 3   trestbps  1025 non-null   int64  
 4   chol      1025 non-null   int64  
 5   fbs       1025 non-null   int64  
 6   restecg   1025 non-null   int64  
 7   thalach   1025 non-null   int64  
 8   exang     1025 non-null   int64  
 9   oldpeak   1025 non-null   float64
 10  slope     1025 non-null   int64  
 11  ca        1025 non-null   int64  
 12  thal      1025 non-null   int64  
 13  target    1025 non-null   int64  
dtypes: float64(1), int64(13)
memory usage: 112.2 KB


None


Basic Statistics:


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,1025.00,1025.00,1025.00,1025.00,1025.00,1025.00,1025.00,1025.00,1025.00,1025.00,1025.00,1025.00,1025.00,1025.00
mean,54.43,0.70,0.94,131.61,246.00,0.15,0.53,149.11,0.34,1.07,1.39,0.75,2.32,0.51
std,9.07,0.46,1.03,17.52,51.59,0.36,0.53,23.01,0.47,1.18,0.62,1.03,0.62,0.50
min,29.00,0.00,0.00,94.00,126.00,0.00,0.00,71.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,48.00,0.00,0.00,120.00,211.00,0.00,0.00,132.00,0.00,0.00,1.00,0.00,2.00,0.00
50%,56.00,1.00,1.00,130.00,240.00,0.00,1.00,152.00,0.00,0.80,1.00,0.00,2.00,1.00
75%,61.00,1.00,2.00,140.00,275.00,0.00,1.00,166.00,1.00,1.80,2.00,1.00,3.00,1.00
max,77.00,1.00,3.00,200.00,564.00,1.00,2.00,202.00,1.00,6.20,2.00,4.00,3.00,1.00



Missing Values:


age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

Total missing values: 0
Total duplicates: 723

Column Names:
['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target']


# 2. ETL Process (Extract, Transform, Load)

In [118]:
# ETL: Clean and transform data before loading into analysis
def perform_etl(data):
    """
    Perform ETL process: Extract, Transform, Load
    Clean and transform data before loading into analysis
    """
    print("=== ETL PROCESS ===")
    
    # Create a copy for ETL
    etl_data = data.copy()
    
    # Step 1: Data Quality Checks
    print("1. Data Quality Checks:")
    print(f"   Initial shape: {etl_data.shape}")
    print(f"   Missing values: {etl_data.isnull().sum().sum()}")
    
    # Handle missing values if any
    if etl_data.isnull().sum().sum() > 0:
        # For numerical columns, fill with median
        numerical_cols = etl_data.select_dtypes(include=[np.number]).columns
        etl_data[numerical_cols] = etl_data[numerical_cols].fillna(etl_data[numerical_cols].median())
        
        # For categorical columns, fill with mode
        categorical_cols = etl_data.select_dtypes(include=['object']).columns
        for col in categorical_cols:
            etl_data[col] = etl_data[col].fillna(etl_data[col].mode()[0])
    
    # Step 2: Data Type Validation and Conversion
    print("2. Data Type Validation and Conversion:")
    print(f"   Data type is correct")

    # Check and convert data types if needed
    for col in etl_data.columns:
        if etl_data[col].dtype == 'object':
            # Try to convert to numeric if possible
            try:
                etl_data[col] = pd.to_numeric(etl_data[col])
                print(f"Converted {col} to numeric")
            except:
                print(f"{col} remains as categorical")

    # Step 3: Remove duplicates
    initial_rows = len(etl_data)
    etl_data = etl_data.drop_duplicates()
    print(f"3. Removed {initial_rows - len(etl_data)} duplicate rows")
    
    # Step 4: Outlier detection and treatment (using IQR method)
    print("4. Outlier Treatment:")
    numerical_cols = etl_data.select_dtypes(include=[np.number]).columns
    outlier_count = 0
    
    for col in numerical_cols:
        Q1 = etl_data[col].quantile(0.25)
        Q3 = etl_data[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        outliers = etl_data[(etl_data[col] < lower_bound) | (etl_data[col] > upper_bound)]
        outlier_count += len(outliers)
        
        # Cap outliers
        etl_data[col] = np.where(etl_data[col] < lower_bound, lower_bound, etl_data[col])
        etl_data[col] = np.where(etl_data[col] > upper_bound, upper_bound, etl_data[col])
    
    print(f"   Treated {outlier_count} outliers across all numerical columns")
    
    # Step 5: Feature Engineering
    print("5. Feature Engineering:")
    print("   Created age groups and cholesterol categories")
    
    # Create age groups
    etl_data['age_group'] = pd.cut(etl_data['age'], bins=[0, 40, 50, 60, 100], labels=['Young', 'Middle-Aged', 'Senior', 'Elderly'])
    
    # Create cholesterol categories
    etl_data['chol_category'] = pd.cut(etl_data['chol'], bins=[0, 200, 240, 1000], labels=['Normal', 'Borderline', 'High'])
    
    print(f"\nFinal ETL dataset shape: {etl_data.shape}")

    return etl_data

# Perform ETL
etl_processed_data = perform_etl(df)
print("\nETL Process Completed Successfully!")

display(etl_processed_data.head())

=== ETL PROCESS ===
1. Data Quality Checks:
   Initial shape: (1025, 14)
   Missing values: 0
2. Data Type Validation and Conversion:
   Data type is correct
3. Removed 723 duplicate rows
4. Outlier Treatment:
   Treated 91 outliers across all numerical columns
5. Feature Engineering:
   Created age groups and cholesterol categories

Final ETL dataset shape: (302, 16)

ETL Process Completed Successfully!


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target,age_group,chol_category
0,52.0,1.0,0.0,125.0,212.0,0.0,1.0,168.0,0.0,1.0,2.0,2.0,3.0,0.0,Senior,Borderline
1,53.0,1.0,0.0,140.0,203.0,0.0,0.0,155.0,1.0,3.1,0.0,0.0,3.0,0.0,Senior,Borderline
2,70.0,1.0,0.0,145.0,174.0,0.0,1.0,125.0,1.0,2.6,0.0,0.0,3.0,0.0,Elderly,Normal
3,61.0,1.0,0.0,148.0,203.0,0.0,1.0,161.0,0.0,0.0,2.0,1.0,3.0,0.0,Elderly,Borderline
4,62.0,0.0,0.0,138.0,294.0,0.0,1.0,106.0,0.0,1.9,1.0,2.5,2.0,0.0,Elderly,High


# 3. ELT Process (Extract, Load, Transform)

In [ ]:
# ELT: Load raw data first, then transform within the sandbox
def perform_elt(data):
    """
    Perform ELT process: Extract, Load, Transform
    Load raw data first, then transform within the sandbox
    """
    print("\n=== ELT PROCESS ===")
    
    # Create a copy for ELT (simulating raw data loading)
    elt_data = data.copy()
    print("1. Raw data loaded into sandbox")
    print(f"   Raw data shape: {elt_data.shape}")
    
    # Step 2: Transform within the sandbox
    print("2. Transforming data within sandbox:")
    
    # Create a transformation log
    transformation_log = []
    
    # Normalize numerical features
    numerical_cols = elt_data.select_dtypes(include=[np.number]).columns
    scaler = StandardScaler()
    
    # Store original values for reference
    for col in numerical_cols:
        elt_data[f'{col}_original'] = elt_data[col]
    
    # Apply standardization
    elt_data[numerical_cols] = scaler.fit_transform(elt_data[numerical_cols])
    transformation_log.append("Standardized all numerical features")
    
    # Create interaction features
    elt_data['age_chol_interaction'] = elt_data['age'] * elt_data['chol']
    elt_data['bp_chol_ratio'] = elt_data['trestbps'] / (elt_data['chol'] + 1)
    transformation_log.append("Created interaction features: age_chol_interaction, bp_chol_ratio")
    
    # Create categorical encodings
    categorical_features = ['cp', 'restecg', 'slope', 'thal']
    for feature in categorical_features:
        if feature in elt_data.columns:
            # One-hot encoding for low cardinality features
            if elt_data[feature].nunique() <= 5:
                dummies = pd.get_dummies(elt_data[feature], prefix=feature)
                elt_data = pd.concat([elt_data, dummies], axis=1)
                transformation_log.append(f"One-hot encoded {feature}")
    
    print("   Transformations applied:")
    for log in transformation_log:
        print(f"     - {log}")
    
    print(f"   Final ELT dataset shape: {elt_data.shape}")
    
    return elt_data, transformation_log

# Perform ELT
elt_processed_data, transformations = perform_elt(df)
print("\nELT Process Completed Successfully!")